In [ ]:
%pip install matplotlib seaborn pandas

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast

In [86]:
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

In [87]:
alunos_df = pd.read_csv("/home/diogo/Documentos/estacio/analise-de-dados/dados/alunos.csv")
responsaveis_df = pd.read_csv("/home/diogo/Documentos/estacio/analise-de-dados/dados/responsavel.csv")

In [ ]:
# Exibe uma prévia dos dados

print ("Previa dos dados dos alunos: ")
display(alunos_df.head())

print("Prévia dos dados dos responsáveis: ")
display(responsaveis_df.head())

In [ ]:
# Converter a coluna 'responsavel_por' de string para listas
responsaveis_df["responsavel_por"] = responsaveis_df["responsavel_por"].apply(ast.literal_eval)

# Expandir a coluna 'responsavel_por' para criar uma relação 1-para-1 entre responsáveis e IDs de alunos
responsaveis_expandido = responsaveis_df.explode("responsavel_por").rename(columns={"responsavel_por": "id_aluno"})

# Realizar o merge entre os DataFrames usando o 'id' do aluno
dados_combinados = pd.merge(
    alunos_df,
    responsaveis_expandido,
    left_on="id",          # ID do aluno no DataFrame de alunos
    right_on="id_aluno"    # ID do aluno no DataFrame expandido de responsáveis
)

# Exibir os dados combinados
print("Dados combinados:")
display(dados_combinados.head())

In [ ]:
# Criação do histograma de idades
plt.figure(figsize=(10, 6))
sns.histplot(alunos_df["idade"], bins=10, kde=True, color="blue")
plt.title("Distribuição de Idades dos Alunos")
plt.xlabel("Idade")
plt.ylabel("Frequência")
plt.show()

# Contagem de graduados por faixa
graduacao_counts = alunos_df["graduacao"].value_counts()

# Gráfico de barras para proporção de graduados
plt.figure(figsize=(10, 6))
sns.barplot(x=graduacao_counts.index, y=graduacao_counts.values, palette="viridis")
plt.title("Proporção de Alunos por Graduação")
plt.xlabel("Graduação")
plt.ylabel("Quantidade")
plt.show()

# Cálculo do número de alunos por responsável
responsaveis_df["quantidade_alunos"] = responsaveis_df["responsavel_por"].apply(len)

# Gráfico de dispersão: Número de alunos x Média Salarial
plt.figure(figsize=(10, 6))
sns.scatterplot(data=responsaveis_df, x="quantidade_alunos", y="media_salarial", hue="mora_no_bairro", palette="cool")
plt.title("Relação entre Número de Alunos e Média Salarial")
plt.xlabel("Número de Alunos")
plt.ylabel("Média Salarial")
plt.legend(title="Mora no Bairro", loc="upper right")
plt.show()

# Estatísticas descritivas com informações adicionais
print("\nEstatísticas descritivas dos responsáveis:")
display(responsaveis_df[["media_salarial", "quantidade_alunos"]].describe())

In [ ]:
# Análise combinada: cálculo de idade média e média salarial por responsável
analise_df = dados_combinados.groupby("id_y").agg({
    "idade": "mean",          # Idade média dos alunos
    "media_salarial": "first" # Média salarial (não muda por responsável)
}).reset_index()

# Renomeia colunas para clareza
analise_df.rename(columns={"idade": "idade_media_alunos", "media_salarial": "media_salarial_responsavel"}, inplace=True)

# Exibição da análise combinada
print("Prévia da análise combinada:")
display(analise_df.head())

# Gráfico de dispersão: Idade Média dos Alunos x Média Salarial dos Responsáveis
plt.figure(figsize=(10, 6))
sns.scatterplot(
    data=analise_df,
    x="idade_media_alunos",
    y="media_salarial_responsavel",
    hue="idade_media_alunos",
    palette="cool"
)
plt.title("Relação entre Idade Média dos Alunos e Média Salarial dos Responsáveis")
plt.xlabel("Idade Média dos Alunos")
plt.ylabel("Média Salarial dos Responsáveis")
plt.show()